In [45]:
import pandas as pd

df = pd.read_csv("index_articles.csv", sep="|", dtype='str')

In [47]:
df

,date,url,title,content,full_text
0,2024/12/31,https://index.hu/gazdasag/2024/12/31/demjan-sa...,"Új kör szekerét tolja meg a kormány, „ingyenpé...",A hazai kis- és középvállalkozások hóna alá ny...,"Új kör szekerét tolja meg a kormány, „ingyenpé..."
1,2024/12/31,https://index.hu/gazdasag/2024/12/31/magyarors...,Financial Times: Magyarország több mint egymil...,"A brit gazdasági szaklap arról írt, hogy Magya...",Financial Times: Magyarország több mint egymil...
2,2024/12/31,https://index.hu/gazdasag/2024/12/31/orban-bal...,Orbán Balázs: A vendégmunkások nem maradnak az...,"Magyarország gazdasági kilátásairól, az eddig ...",Orbán Balázs: A vendégmunkások nem maradnak az...
3,2024/12/31,https://index.hu/gazdasag/2024/12/31/uzemanyag...,Túladóztat a kormány a szakértő szerint – ez v...,Szerdától emelkedik a benzin és a gázolaj nagy...,Túladóztat a kormány a szakértő szerint – ez v...
4,2024/12/31,https://index.hu/gazdasag/2024/12/31/penzugyek...,Januártól záporoznak a fontos pénzügyi változá...,"Felsorolni is nehéz, mennyi változás történik ...",Januártól záporoznak a fontos pénzügyi változá...
...,...,...,...,...,...
26096,2018/01/02,https://index.hu/gazdasag/2018/01/02/fellebbez...,Fellebbez a Paks II.-ből kizárt orosz cég,Kifogást nyújtott be az orosz Szilovije Masini...,Fellebbez a Paks II.-ből kizárt orosz cég. Kif...
26097,2018/01/02,https://index.hu/gazdasag/2018/01/02/5_milliar...,5 milliárd forintnyi EU-s pénzt nyert az NGM c...,Újra nekifuthat a kormány a közszféra leépítés...,5 milliárd forintnyi EU-s pénzt nyert az NGM c...
26098,2018/01/02,https://index.hu/gazdasag/2018/01/02/sajat_fut...,Saját futárszolgáltatot indít az Edigital,Saját futárszolgálat indítását és első külföld...,Saját futárszolgáltatot indít az Edigital. Saj...
26099,2018/01/01,https://index.hu/gazdasag/2018/01/01/igenis_ol...,Igenis olcsók a budapesti lakások,Frissebb (1) Vége az őrületnek a magyar lakásp...,Igenis olcsók a budapesti lakások. Frissebb (1...


In [9]:
df = df[(df["title"] != "Title not found") & ~df["content"].str.contains("Content not found") & ~df["content"].str.strip().isna() & ~df["content"].str.contains("Error fetching content")]

In [11]:
df = df.reset_index(drop=True)

In [31]:
df.to_csv("index_articles.csv", sep="|", encoding='utf-8', index=False)

In [29]:
df = df.drop("Unnamed: 0", axis=1)

In [20]:
df['full_text'] = df['title'] + ". " + df['content']

In [25]:
df_test = df.head(400)

In [36]:
import pandas as pd
from google import genai
from google.genai import types
import time

client = genai.Client(api_key="")

# Function to translate each text
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Load existing results if available (to resume progress)
try:
    df_translated = pd.read_csv("translated_articles_index.csv")
    processed_ids = set(df_translated.index)
except FileNotFoundError:
    df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
    processed_ids = set()

# Batch processing with periodic saving
batch_size = 100  # Number of rows to process per batch
start_index = max(processed_ids) + 1 if processed_ids else 0

for i in range(start_index, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    
    # Translate each article in the batch
    batch["translated_content"] = batch["full_text"].apply(translate_text)
    
    # Append to the result DataFrame
    df_translated = pd.concat([df_translated, batch], ignore_index=True)
    
    # Save to CSV after each batch
    df_translated.to_csv("translated_articles_index.csv", index=False, sep="|", encoding='utf-8')
    
    print(f"Batch {i // batch_size + 1} processed and saved. Total articles translated: {len(df_translated)}")
    time.sleep(2)  # Add a small delay to avoid hitting API rate limits


C:\Users\tothk\AppData\Local\Temp\ipykernel_14904\3004148015.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = batch["full_text"].apply(translate_text)


Batch 1 processed and saved. Total articles translated: 100


C:\Users\tothk\AppData\Local\Temp\ipykernel_14904\3004148015.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = batch["full_text"].apply(translate_text)


Batch 2 processed and saved. Total articles translated: 200


C:\Users\tothk\AppData\Local\Temp\ipykernel_14904\3004148015.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = batch["full_text"].apply(translate_text)


...

C:\Users\tothk\AppData\Local\Temp\ipykernel_14904\3004148015.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = batch["full_text"].apply(translate_text)


Batch 66 processed and saved. Total articles translated: 6600
Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


C:\Users\tothk\AppData\Local\Temp\ipykernel_14904\3004148015.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = batch["full_text"].apply(translate_text)


Batch 67 processed and saved. Total articles translated: 6700


KeyboardInterrupt: 

In [49]:
df_proba = pd.read_csv("translated_articles_index.csv", sep="|", dtype='str')

In [14]:
df_proba[df_proba["translated_content"].isnull()].count()

full_text             283
translated_content      0
date                  283
url                   283
title                 283
content               283
dtype: int64

In [48]:
import pandas as pd
import asyncio
from concurrent.futures import ThreadPoolExecutor
from google import genai
from google.genai import types

client = genai.Client(api_key="")

# Function to translate each text 
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Asynchronous wrapper for the translate function
async def translate_batch_async(batch):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, translate_text, text) for text in batch["full_text"]]
        results = await asyncio.gather(*tasks)
    batch["translated_content"] = results
    return batch

# Main processing loop
async def main(df):
    try:
        df_translated = pd.read_csv("translated_articles_index.csv", sep="|", encoding='utf-8')
        processed_ids = set(df_translated.index)
    except FileNotFoundError:
        df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
        processed_ids = set()

    batch_size = 75  # Number of rows to process per batch
    start_index = max(processed_ids) + 1 if processed_ids else 0

    for i in range(start_index, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        batch = await translate_batch_async(batch)
        
        # Append to the result DataFrame
        df_translated = pd.concat([df_translated, batch], ignore_index=True)
        
        # Save to CSV after each batch
        df_translated.to_csv("translated_articles_index.csv", index=False, sep="|", encoding='utf-8')
        
        print(f"Batch {i // batch_size + 1} processed and saved. Total articles translated: {len(df_translated)}")


# Run the asynchronous main function
await main(df)


C:\Users\tothk\AppData\Local\Temp\ipykernel_14904\564932478.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 90 processed and saved. Total articles translated: 6775


C:\Users\tothk\AppData\Local\Temp\ipykernel_14904\564932478.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 91 processed and saved. Total articles translated: 6850


C:\Users\tothk\AppData\Local\Temp\ipykernel_14904\564932478.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


...

Batch 346 processed and saved. Total articles translated: 25975


C:\Users\tothk\AppData\Local\Temp\ipykernel_14904\564932478.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 347 processed and saved. Total articles translated: 26050


C:\Users\tothk\AppData\Local\Temp\ipykernel_14904\564932478.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 348 processed and saved. Total articles translated: 26101


In [11]:
import pandas as pd
import asyncio
from concurrent.futures import ThreadPoolExecutor
from google import genai
from google.genai import types

client = genai.Client(api_key="")

# Function to translate each text 
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Asynchronous wrapper for the translate function
async def translate_batch_async(batch):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, translate_text, text) for text in batch["full_text"]]
        results = await asyncio.gather(*tasks)
    batch["translated_content"] = results
    return batch

# Main function to process and retry NaN translations
async def main():
    try:
        df_translated = pd.read_csv("translated_articles_index.csv", sep="|", encoding='utf-8')
    except FileNotFoundError:
        df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
    
    # First pass: Process untranslated rows
    untranslated_rows = df_translated["translated_content"].isna().sum()
    print(f"Untranslated rows found: {untranslated_rows}")

    if untranslated_rows > 0:
        to_retry = df_translated[df_translated["translated_content"].isna()]
        df_translated.loc[to_retry.index, "translated_content"] = (await translate_batch_async(to_retry))["translated_content"].values

        # Save the corrected translations
        df_translated.to_csv("translated_articles_index.csv", index=False, sep="|", encoding='utf-8')
        print(f"Retry complete! Remaining untranslated rows: {df_translated['translated_content'].isna().sum()}")
    
    print("Translation process complete!")

# Run the asynchronous main function
await main()


Untranslated rows found: 350


C:\Users\tothk\AppData\Local\Temp\ipykernel_14640\507195572.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Retry complete! Remaining untranslated rows: 283
Translation process complete!


In [12]:
import pandas as pd

df_proba = pd.read_csv("translated_articles_index.csv", sep="|", dtype='str')

In [13]:
df_proba

,full_text,translated_content,date,url,title,content
0,"Új kör szekerét tolja meg a kormány, „ingyenpé...","The government is launching a new initiative, ...",2024/12/31,https://index.hu/gazdasag/2024/12/31/demjan-sa...,"Új kör szekerét tolja meg a kormány, „ingyenpé...",A hazai kis- és középvállalkozások hóna alá ny...
1,Financial Times: Magyarország több mint egymil...,Financial Times: Hungary Could Lose Over a Bil...,2024/12/31,https://index.hu/gazdasag/2024/12/31/magyarors...,Financial Times: Magyarország több mint egymil...,"A brit gazdasági szaklap arról írt, hogy Magya..."
2,Orbán Balázs: A vendégmunkások nem maradnak az...,**Orbán Balázs: Guest Workers Will Not Stay in...,2024/12/31,https://index.hu/gazdasag/2024/12/31/orban-bal...,Orbán Balázs: A vendégmunkások nem maradnak az...,"Magyarország gazdasági kilátásairól, az eddig ..."
3,Túladóztat a kormány a szakértő szerint – ez v...,"According to an expert, the government is over...",2024/12/31,https://index.hu/gazdasag/2024/12/31/uzemanyag...,Túladóztat a kormány a szakértő szerint – ez v...,Szerdától emelkedik a benzin és a gázolaj nagy...
4,Januártól záporoznak a fontos pénzügyi változá...,"From January, important financial changes will...",2024/12/31,https://index.hu/gazdasag/2024/12/31/penzugyek...,Januártól záporoznak a fontos pénzügyi változá...,"Felsorolni is nehéz, mennyi változás történik ..."
...,...,...,...,...,...,...
26096,Fellebbez a Paks II.-ből kizárt orosz cég. Kif...,The Russian company excluded from Paks II. is ...,2018/01/02,https://index.hu/gazdasag/2018/01/02/fellebbez...,Fellebbez a Paks II.-ből kizárt orosz cég,Kifogást nyújtott be az orosz Szilovije Masini...
26097,5 milliárd forintnyi EU-s pénzt nyert az NGM c...,The National Economy Ministry (NGM) has won 5 ...,2018/01/02,https://index.hu/gazdasag/2018/01/02/5_milliar...,5 milliárd forintnyi EU-s pénzt nyert az NGM c...,Újra nekifuthat a kormány a közszféra leépítés...
26098,Saját futárszolgáltatot indít az Edigital. Saj...,Edigital is launching its own courier service....,2018/01/02,https://index.hu/gazdasag/2018/01/02/sajat_fut...,Saját futárszolgáltatot indít az Edigital,Saját futárszolgálat indítását és első külföld...
26099,Igenis olcsók a budapesti lakások. Frissebb (1...,"Yes, apartments in Budapest are cheap.\n\n**Is...",2018/01/01,https://index.hu/gazdasag/2018/01/01/igenis_ol...,Igenis olcsók a budapesti lakások,Frissebb (1) Vége az őrületnek a magyar lakásp...


In [15]:
df_proba = pd.read_csv("translated_articles_index.csv", sep="|", dtype='str')

df_proba.dropna(subset=['translated_content'], inplace=True)
df_proba.reset_index(drop=True, inplace=True)

df_proba.to_csv("translated_articles_index.csv", index=False, sep="|", encoding='utf-8')